# Programación Python para Machine Learning.

## Autor: Victor Simo Lozano

## Actividad 11
<p>Ajuste de hiperparámetros y flujos de trabajo en Machine Learning.</p>
    

<hr style="border-color:red">

In [1]:
# Generales 
from pandas import pandas as pd
from scipy.io import arff
import numpy as np
# import matplotlib.pyplot as plt
import random
from time import time
# from IPython.display import clear_output

# Seleccion de caracteristicas
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
# # from sklearn.feature_selection import SelectKBest
# # from sklearn.feature_selection import f_classif
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
# # from sklearn.metrics import plot_confusion_matrix
# # from sklearn.utils import resample
# from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.impute import SimpleImputer

# Metricas
# from sklearn.metrics import mean_squared_error

# Calculo de modelos
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, KFold
# from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor

In [2]:
import warnings
warnings.filterwarnings('ignore')

Selecciona un conjunto de datos de los múltiples repositorios existentes correspondiente a un problema de clasificación múltiple y realiza los siguientes apartados:
- Implementa un flujo de trabajo que incluya:
    - Imputación de valores perdidos.
    - Normalización de datos.
    - Ajuste de hiperparametros mediante búsqueda grid.
    - Entrenamiento de una red neuronal mediante validación cruzada.
<br>
- Finalmente, reporta la mejor combinación posible de hiperparámetros y la correspondiente métrica de evaluación de su rendimiento.


***Fuente:*** <br>
*https://www.kaggle.com/datasets/ppsheth91/two-target-variables-classification-problem*

<div style="font-size:14px; text-align:justify"><b>PRIMERA PARTE.-</b><br></div>

In [3]:
seed=random.randint(0,100)

# Carga del dataset
df=pd.read_csv('train.csv', sep=',')

# Separación en datos de entrada y salida del dataframe
X=df[df.columns[:-1]]
# y=df[df.columns[-1:]]
y=df['pet_category']

display(X.sample(3), y.sample(3))

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category
18205,ANSL_56709,2016-12-24 00:00:00,2017-03-17 18:12:00,1.0,Black,0.54,29.95,0,1,0.0
6895,ANSL_59675,2016-12-23 00:00:00,2019-01-14 17:13:00,2.0,Brown,0.13,18.90,0,7,1.0
4245,ANSL_59248,2015-08-22 00:00:00,2017-09-19 18:32:00,0.0,Tan,0.62,24.78,0,1,1.0


9673     1
805      2
12647    2
Name: pet_category, dtype: int64

In [4]:
y.value_counts()

2    10621
1     7184
4      941
0       88
Name: pet_category, dtype: int64

Como se puede observar, se trata de un conjunto de datos muy desbalanceado.

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
dtypes: float64(4), int64(2), object(4)
memory usage: 1.4+ MB


Visualizando la información de los datos, se observa que para el atributo *"condition"* existe valores faltantes. Además, esta columna así como *"breed_category"* son numéricas, y en realidad, las mismas categóricas por lo que será necesario realizar un One Hot Encoding para su uso correcto.

In [6]:
X['condition'].value_counts()

1.0    6819
0.0    6281
2.0    4257
Name: condition, dtype: int64

Primeramente, generamos dos clases que serán empleadas para el tratamiento de los datos mediante el uso de Pipelines con sus metodos *fit* y *transform*.

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

# Clase creada para usar en Pipeline que permitirá eliminar las columnas deseadas
class Column_Dropper(BaseEstimator, TransformerMixin):
    
    def __init__(self,columns):
        self.columns=columns

    def transform(self,X,y=None):
        X_=X.copy()
        X_=X_.drop(self.columns,axis=1)
        self.actual_columns = X_.columns
        # print(list(self.actual_columns))
        return X_

    def fit(self, X, y=None):
        return self 
    
    def get_feature_names(self, X):
        # print(list(self.actual_columns))
        return list(self.actual_columns)
    

# Clase creada para convertir tipo de columna.
class Column_Casting():
    
    def __init__(self,feature):
        self.feature_name=feature
    
    def fit(self, X, y=None):  
            
        return self
    
    def transform(self, X, y=None):
        X_=X.copy()
        for feature_name in self.feature_name: 
            X_[feature_name]=X_[feature_name].astype('object')
            
        return X_

In [8]:
# Inicio del procesamiento de datos con Pipeline.
start_time=time()

# Pipeline inicial para el preprocesamiento de los datos:
# 1: Eliminar columnas indeseadas.
# 2: Formatear el tipo de columna deseado.

pre_proccess=Pipeline(steps=[
    ('column_Dropper', Column_Dropper(['pet_id','issue_date','listing_date'])),
    ('casting1', Column_Casting(['condition'])),
    ('casting2', Column_Casting(['breed_category']))
])
# pre_proccess.fit_transform(X)


# Pipeline para el tratamiento de los datos:
# 1: Imputación de valores perdidos.
# 2: Normalización de los datos.
# ColumnComposer para el tratado de diferentes columnas del dataframe que despues se unirán en un PipeLine

numerical_features=['length(m)','height(cm)','X1','X2']
categorical_features=['condition','color_type','breed_category']
impute_features=['condition']

impute_values=Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='0'))])

scaler=Pipeline(steps=[('scaler', RobustScaler())])

ohe_transform=Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))])
          
    
# Union de todos los pipelines empleados y creación del Pipeline principal con el modelo de Red Neuronal

preprocessor=ColumnTransformer([
    # ('data_proccess', pre_proccess, list(X.columns)),
    ('imputing', impute_values, impute_features),
    ('scaling', scaler, numerical_features),
    ('encoding', ohe_transform, categorical_features)
])


pipe = Pipeline([
    ('data_proccess', pre_proccess),
    ('preprocessor', preprocessor),
    ('MLPClassifier', MLPClassifier(random_state=seed))
])


# Ajuste parametros GridSearchCV 

param_grid = {
    # "MLPClassifier__solver": ['sgd','adam'],
    "MLPClassifier__max_iter": [2000,3000],
    "MLPClassifier__activation": ['relu','logistic','tanh'],
    "MLPClassifier__hidden_layer_sizes": [(10,10),(20,)],
}

# cross_validation=RepeatedStratifiedKFold(n_splits=7,n_repeats=2,random_state=seed)
cross_validation=KFold(n_splits=7, shuffle=True, random_state=seed)

grid_search=GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=cross_validation,
    return_train_score=True,
    scoring='balanced_accuracy',
    # n_jobs=-1
)

result=grid_search.fit(X, y)
# result = pipe.fit(X, y)

display(pipe)

# Finalización del modelo
total_time=time()-start_time

print(f'''Tiempo empleado {total_time:.2f} (s)''')


Pipeline(steps=[('data_proccess',
                 Pipeline(steps=[('column_Dropper',
                                  Column_Dropper(columns=['pet_id',
                                                          'issue_date',
                                                          'listing_date'])),
                                 ('casting1',
                                  <__main__.Column_Casting object at 0x000001E30F9B31F0>),
                                 ('casting2',
                                  <__main__.Column_Casting object at 0x000001E30F9B3220>)])),
                ('preprocessor',
                 ColumnTransformer(transformers=[('imputing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant'))]),
                                                  ['condition']),
                                                 ('scaling',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['length(m)', 'height(cm)',
                                                   'X1', 'X2']),
                                                 ('encoding',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['condition', 'color_type',
                                                   'breed_category'])])),
                ('MLPClassifier', MLPClassifier(random_state=28))])

Tiempo empleado 1045.25 (s)


In [15]:
print(f'''Accuracy obtenida: {result.best_score_:.3f}''')
print(f'''Balanced Score obtenida: {result.score(X, y):.3f}''')
print(f'''Mejores parámetros para el modelo: {result.best_params_}''')

Accuracy obtenida: 0.672
Balanced Score obtenida: 0.710
Mejores parámetros para el modelo: {'MLPClassifier__activation': 'tanh', 'MLPClassifier__hidden_layer_sizes': (20,), 'MLPClassifier__max_iter': 2000}
